In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
# Loading the cleaned dataset, in which within-mouse outliers have been removed and then the MEDIAN per mouse has been calculated
# Outlier biological replicates are still present.  Some mice are missing data for some proteins.
# (8/76 proteins are missing at least one data point.)
data = pd.read_csv('Data_Cortex_Nuclear_Medians_Clean.csv')
data.replace(to_replace={'C/S': 'Learning', 'S/C': 'Control'}, inplace=True)
data.head()

,Genotype,Behavior,Treatment,class,Mouse,DYRK1A_N,ITSN1_N,BDNF_N,NR1_N,NR2A_N,...,pGSK3B_Tyr216_N,SHH_N,BAD_N,BCL2_N,pCFOS_N,SYP_N,H3AcK18_N,EGR1_N,H3MeK4_N,CaNA_N
0,Control,Learning,Memantine,c-CS-m,309,0.416923,0.564036,0.355124,2.260135,4.268735,...,0.833277,0.191561,0.141013,NaN,0.111483,0.434154,0.130405,0.155612,0.158235,1.675652
1,Control,Learning,Memantine,c-CS-m,311,0.591572,0.690271,0.320853,2.285938,4.229412,...,0.679241,0.181889,0.167400,0.168637,NaN,0.430309,0.147208,0.215089,0.197067,1.219966
2,Control,Learning,Memantine,c-CS-m,320,0.530855,0.748072,0.385529,2.606780,5.601210,...,0.854093,0.220316,0.161053,0.136110,0.123425,0.534845,0.143169,0.180373,NaN,1.638988
3,Control,Learning,Memantine,c-CS-m,321,0.420542,0.640420,0.357512,2.731909,5.468987,...,0.922769,0.234360,NaN,0.160967,NaN,0.561673,0.152119,0.214380,NaN,1.682427
4,Control,Learning,Memantine,c-CS-m,322,0.356514,0.500887,0.316709,2.062867,4.128476,...,0.829548,0.239625,NaN,0.150934,0.140960,0.494885,NaN,NaN,NaN,1.614975


In [20]:
# identify which columns represent protein names
not_a_protein = ['Genotype','Behavior','Treatment','class', 'Mouse']
protein_names = data.columns[5:]
X_columns = data[protein_names]
medians = X_columns.median()
X_imputed = X_columns.fillna(medians)
Y = data[['Genotype','Behavior','Treatment','class']]
y_geno  = data['Genotype']
y_behav = data['Behavior']
y_treat = data['Treatment']
y_class = data['class']

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble        import RandomForestClassifier
from sklearn.ensemble        import GradientBoostingClassifier
from sklearn.neighbors       import NearestNeighbors
from sklearn.ensemble        import VotingClassifier

def default_rf():
    return RandomForestClassifier(criterion = 'gini', max_features = 'sqrt', n_estimators = 1000, 
                                       max_depth = None, n_jobs = -1, min_samples_leaf = 1)

def default_gb():
    return GradientBoostingClassifier(n_estimators = 1000, subsample = 0.8, max_depth = 10)

def default_nn():
    return 

def fit_and_print_accuracy(classifier, X_train, y_train, X_test, y_test):
    classifier.fit(X_train, y_train)
    training_accuracy = classifier.score(X_train, y_train)
    test_accuracy = classifier.score(X_test, y_test)
    print("Accuracy on training data: {:2f}".format(training_accuracy))
    print("Accuracy on test data:     {:2f}".format(test_accuracy))

In [21]:
X_train, X_test, Y_train, Y_test = train_test_split(X_imputed, Y, 
                                                    test_size = 0.2, stratify = y_class)


In [6]:
#importance_df = pd.DataFrame({'protein': X_columns.columns, 
#                              'importance': classifier_rf.feature_importances_})
#importance_df.sort_values('importance', ascending = False).head()

In [24]:
results = {'variable': [], 
           'rf_train': [],
           'rf_test': [], 
           'gb_train': [], 
           'gb_test': []}
for variable in Y.columns:
    results['variable'].append(variable)
    y_train = Y_train[variable]
    y_test =  Y_test[variable]
    classifier_rf = default_rf()
    classifier_rf.fit(X_train, y_train)
    results['rf_train'].append(classifier_rf.score(X_train, y_train))
    results['rf_test'].append(classifier_rf.score(X_test, y_test))
    classifier_gb = default_gb()
    classifier_gb.fit(X_train, y_train)
    results['gb_train'].append(classifier_gb.score(X_train, y_train))
    results['gb_test'].append(classifier_gb.score(X_test, y_test))
    
pd.DataFrame(results)

,variable,rf_train,rf_test,gb_train,gb_test
0,Genotype,1.0,0.733333,1.0,0.800000
1,Behavior,1.0,0.933333,1.0,1.000000
2,Treatment,1.0,0.666667,1.0,0.666667
3,class,1.0,0.600000,1.0,0.533333


In [62]:
def combine_classifier_predict(classifier_list, X):
    #Assumes classifiers have already been fit to training data
    probabilites = []
    predictions = []
    for classifier in classifier_list:
        probabilites.append(classifier.predict_proba(X))
        predictions.append(classifier.predict(X))
    best_predictions = []
    for x_i in range(len(X)):
        x_predictions = []
        x_probabilities = []
        for c_i in range(len(classifier_list)):
            x_predictions.append(predictions[c_i][x_i])
            x_probabilities.append(np.max(probabilites[c_i][x_i]))
        print(x_predictions, x_probabilities)
        best_predictions.append(x_predictions[x_probabilities.index(max(x_probabilities))])
    return best_predictions

In [65]:
combine = combine_classifier_predict([classifier_rf, classifier_gb], X_test)
np.mean(combine == y_test)
y_test

['c-SC-m', 'c-SC-m'] [0.387, 0.8470935063056436]
['t-CS-m', 't-CS-m'] [0.256, 0.733947698493443]
['t-CS-s', 't-CS-s'] [0.342, 0.9959919150352845]
['c-SC-m', 'c-SC-m'] [0.438, 0.4464756927996524]
['t-CS-s', 't-CS-s'] [0.259, 0.8528143091408102]
['c-CS-m', 't-CS-m'] [0.25, 0.5519175448242831]
['c-SC-m', 'c-SC-m'] [0.391, 0.9829792939499776]
['t-SC-s', 't-SC-s'] [0.296, 0.34816844438835753]
['t-CS-m', 't-SC-s'] [0.391, 0.8818082748345785]
['c-SC-s', 'c-SC-s'] [0.472, 0.9867429739728306]
['c-CS-s', 't-CS-m'] [0.438, 0.9892178391836185]
['t-SC-s', 't-SC-s'] [0.469, 0.9688888095903124]
['c-CS-m', 'c-CS-m'] [0.371, 0.7774057082864604]
['c-SC-s', 'c-SC-s'] [0.291, 0.5533925176940884]
['t-SC-s', 't-SC-s'] [0.32, 0.7715823918007609]


62    t-SC-m
40    t-CS-m
49    t-CS-s
28    c-SC-m
18    c-CS-s
10    c-CS-s
20    c-SC-m
65    t-SC-s
46    t-CS-m
33    c-SC-s
0     c-CS-m
63    t-SC-s
4     c-CS-m
55    t-SC-m
37    c-SC-s
Name: class, dtype: object

In [64]:
pd.DataFrame(classifier_gb.predict_proba(X_test))

,0,1,2,3,4,5,6,7
0,0.019065,0.023776,0.847094,0.027705,0.014063,0.016262,0.032932,0.019104
1,0.008695,0.004627,0.015839,0.016563,0.733948,0.069952,0.002489,0.147886
2,0.000137,0.000790,0.000108,0.000144,0.002039,0.995992,0.000090,0.000700
3,0.084288,0.088434,0.446476,0.094926,0.062224,0.071915,0.067164,0.084573
4,0.074875,0.054359,0.001454,0.001038,0.002084,0.852814,0.011971,0.001405
5,0.130238,0.173750,0.080511,0.010796,0.551918,0.011604,0.026895,0.014288
6,0.000191,0.000145,0.982979,0.000950,0.000148,0.000119,0.011652,0.003816
7,0.136046,0.105311,0.081542,0.079539,0.061437,0.092318,0.095638,0.348168
8,0.007741,0.004608,0.001995,0.000732,0.051418,0.051081,0.000617,0.881808
9,0.007604,0.000051,0.005418,0.986743,0.000036,0.000042,0.000046,0.000060


In [81]:
pred_df = pd.DataFrame()
for variable in Y.columns[0:3]:
    y_train = Y_train[variable]
    y_test =  Y_test[variable]
    classifier_rf = default_rf()
    classifier_rf.fit(X_train, y_train)
    pred_df[variable] = classifier_rf.predict(X_test)

def interpret_class(genotype, behavior, treatment):
    if (genotype == 'Control'):
        c = 'c-'
    else:
        c = 't-'    
    if (behavior == 'Control'):
        c += 'SC-'
    else:
        c += 'CS-'
    if (treatment == 'Saline'):
        c += 's'
    else:
        c += 'm'
    return c

In [90]:
class_pred = [interpret_class(row[1], row[2], row[3]) for row in pred_df.itertuples()]
print('Accuracy from combining binary predictions is %.2f' %(np.mean(Y_test['class'] == class_pred)))

Accuracy from combining binary predictions is 0.60


In [91]:
class_pred

['c-SC-m',
 't-CS-m',
 't-CS-s',
 'c-SC-m',
 't-CS-m',
 't-CS-m',
 'c-SC-m',
 't-CS-s',
 't-CS-m',
 'c-SC-m',
 'c-CS-m',
 't-SC-s',
 'c-CS-m',
 'c-SC-s',
 'c-SC-s']

In [92]:
Y_test['class']

62    t-SC-m
40    t-CS-m
49    t-CS-s
28    c-SC-m
18    c-CS-s
10    c-CS-s
20    c-SC-m
65    t-SC-s
46    t-CS-m
33    c-SC-s
0     c-CS-m
63    t-SC-s
4     c-CS-m
55    t-SC-m
37    c-SC-s
Name: class, dtype: object